<a href="https://colab.research.google.com/github/yujie0124/Assignment_Address/blob/main/Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk
!pip install Spylls
!pip install malaya

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, regexp_tokenize, treebank
from nltk.util import ngrams
import re
import logging
logging.basicConfig(level=logging.INFO)
import malaya

class Address:
    def __init__(self, user_input):
      self.value = user_input.title()
      self.apt = None
      self.city = None
      self.state = None
      self.postcode = None
      self.street = None
      self.section = None
        
    def getApt(self):
      match = re.search('No.[0-9]+', self.value)
      if match is not None:
        self.apt = match.group()
        self.value = self.value.replace(self.apt, "")
    
    def getPostcode(self):
      match = re.search('[0-9]{5}', self.value)
      if match is not None:
        self.postcode = match.group()
        self.value = self.value.replace(self.postcode, "")
    
    def getCity(self):
      cities = 'Kuala Terengganu|Kuala Lumpur|Kajang|Bangi|Damansara|Petaling Jaya|Puchong|Subang Jaya|Cyberjaya|Putrajaya|Mantin|Kuching|Seremban'
      match = re.search(cities, self.value)
      if match is not None:
        self.city = match.group()
        self.value = self.value.replace(self.city, "")
    
    def getState(self):
      states = 'Selangor|Terengganu|Pahang|Kelantan|Melaka|Pulau Pinang|Kedah|Johor|Perlis|Sabah|Sarawak'
      match = re.search(states, self.value)
      if match is not None:
        self.state = match.group()
        self.value = self.value.replace(self.state, "") 
    
    def getStreet(self):
      matchStreet = re.search('(Jalan|Jln|Lorong|Persiaran)\s[A-Za-z0-9\-\.\/\s]*', self.value)
      if matchStreet is not None:
        self.street = matchStreet.group()
        self.value = self.value.replace(self.street, "")
        
      # contain no Jalan, then all considered as SECTION
      clean = re.sub(r'[\,\.]', '', self.value)
      if clean != '' and clean !=' ':
        self.section = clean
    
    def checkPostcode(self):
      match = re.search('[0-9]+', self.value)
      if match is not None:
        clean = re.sub(r'No.[0-9]+', '', self.value)
        match2 = re.search('[0-9]+', clean)
        if match2 is not None:
          inputPostcode = match2.group()
          if len(inputPostcode) != 5 or (int(inputPostcode) not in range(1000,98860)):
            return False
      else:
        return True
    
    def checkCityState(self):
      clean = re.sub(r'[^A-Za-z\s]+', '', self.value)
      tokens = word_tokenize(clean)
      checker = ["Kuala", "Terengganu", "Lumpur", "Kajang", "Bangi", "Damansara", "Petaling", "Jaya", "Puchong", "Subang", "Cyberjaya", 
                 "Putrajaya", "Mantin", "Kuching", "Seremban", "Selangor", "Pahang", "Kelantan", "Melaka", "Pulau", "Pinang", "Kedah", 
                 "Johor", "Perlis", "Sabah", "Sarawak"]
      x = []
      model = malaya.spelling_correction.spylls.load()
      for word in tokens:
        corr = model.correct(word)
        if word != corr and corr in checker:
            x.append([word, corr.title()])
      return x

    def getComponent(self):
      self.getApt()
      self.getCity()
      self.getState()
      self.getPostcode()
      self.getStreet()
      for attribute, value in self.__dict__.items():
        if value is not None and attribute!="value":
          print(attribute, ':', value, "\n")

In [3]:
while True:
    address = input("Please enter your address: ")
    if address == '':
        print("Invalid data, please enter valid address. Thanks.\n")
        continue
    else:
        ad = Address(address)
        if ad.checkPostcode() == False:
          print("Invalid postcode, please enter valid address. Thanks.\n")
          continue
        x = ad.checkCityState()
        if len(x) != 0:
          for y in x:
            print("You may have entered wrong name of the city or state, '{}'. Perhaps the correct word is '{}'".format(y[0], y[1]))
          continue
        else:
          ad.getComponent()
          cont = input("Wish to continue? (y/n)").lower()
          if cont == 'y':
            print("\n")
            continue
          else:
            break

Please enter your address: No 11, Chendering, 21080 Kuala Terengganu, Terengganu.
apt : No 11 

city : Kuala Terengganu 

state : Terengganu 

postcode : 21080 

section :  Chendering    

Wish to continue? (y/n)y


Please enter your address: No 11 Chendering 21080 Kuala Terengganu Terengganu.
apt : No 11 

city : Kuala Terengganu 

state : Terengganu 

postcode : 21080 

section :  Chendering    

Wish to continue? (y/n)y


Please enter your address: No 11, Kuala Terengganu, Chendering
apt : No 11 

city : Kuala Terengganu 

section :   Chendering 

Wish to continue? (y/n)y


Please enter your address: No 35 Jalan Awan Besar Kuala Lumpur 50400
apt : No 35 

city : Kuala Lumpur 

postcode : 50400 

street : Jalan Awan Besar   

Wish to continue? (y/n)y


Please enter your address: No 35 Jalan Awan Besar Kuala Lumpur 5040
Invalid postcode, please enter valid address. Thanks.

Please enter your address: No 35 Jalan Awan Besar Selangr 50400
You may have entered wrong name of the city or s